In [4]:
# Install necessary packages
!pip install scikit-learn hyperopt

In [5]:


import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK

# Load dataset
data = load_iris()
X = data.data
y = data.target

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [6]:
# Grid Search
# Define parameter grid
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10]
}

# Initialize RandomForestClassifier
rf = RandomForestClassifier(random_state=42)

# Initialize GridSearchCV
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2)

# Fit GridSearchCV
grid_search.fit(X_train, y_train)

# Best parameters and score
print("Best parameters found by GridSearchCV:", grid_search.best_params_)
print("Best accuracy found by GridSearchCV:", grid_search.best_score_)


Fitting 5 folds for each of 36 candidates, totalling 180 fits
Best parameters found by GridSearchCV: {'max_depth': None, 'min_samples_split': 2, 'n_estimators': 200}
Best accuracy found by GridSearchCV: 0.95


In [7]:
# Random Search
# Define parameter distribution
param_dist = {
    'n_estimators': [int(x) for x in np.linspace(start=50, stop=200, num=10)],
    'max_depth': [None] + [int(x) for x in np.linspace(10, 30, num=10)],
    'min_samples_split': [2, 5, 10, 15]
}

# Initialize RandomizedSearchCV
random_search = RandomizedSearchCV(estimator=rf, param_distributions=param_dist, n_iter=100, cv=5, n_jobs=-1, verbose=2, random_state=42)

# Fit RandomizedSearchCV
random_search.fit(X_train, y_train)

# Best parameters and score
print("Best parameters found by RandomizedSearchCV:", random_search.best_params_)
print("Best accuracy found by RandomizedSearchCV:", random_search.best_score_)


Fitting 5 folds for each of 100 candidates, totalling 500 fits
Best parameters found by RandomizedSearchCV: {'n_estimators': 133, 'min_samples_split': 10, 'max_depth': 21}
Best accuracy found by RandomizedSearchCV: 0.95


In [8]:
# Bayesian Optimization
# Define the objective function
def objective(params):
    clf = RandomForestClassifier(**params, random_state=42)
    clf.fit(X_train, y_train)
    pred = clf.predict(X_test)
    accuracy = accuracy_score(y_test, pred)
    return {'loss': -accuracy, 'status': STATUS_OK}

# Define the hyperparameter space
space = {
    'n_estimators': hp.choice('n_estimators', [50, 100, 200]),
    'max_depth': hp.choice('max_depth', [None, 10, 20, 30]),
    'min_samples_split': hp.choice('min_samples_split', [2, 5, 10])
}

# Initialize Trials object
trials = Trials()

# Run Bayesian Optimization
best = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=100, trials=trials, rstate=np.random.default_rng(42))

print("Best parameters found by Bayesian Optimization:", best)


100%|██████████| 100/100 [00:19<00:00,  5.24trial/s, best loss: -1.0]
Best parameters found by Bayesian Optimization: {'max_depth': 2, 'min_samples_split': 2, 'n_estimators': 0}


In [9]:
# Evaluation
# Get the best models
best_grid_model = grid_search.best_estimator_
best_random_model = random_search.best_estimator_

# Train and evaluate the best models
best_grid_model.fit(X_train, y_train)
best_random_model.fit(X_train, y_train)

grid_pred = best_grid_model.predict(X_test)
random_pred = best_random_model.predict(X_test)

print("Test accuracy with GridSearchCV:", accuracy_score(y_test, grid_pred))
print("Test accuracy with RandomizedSearchCV:", accuracy_score(y_test, random_pred))

# Bayesian Optimization result
bayes_params = {
    'n_estimators': [50, 100, 200][best['n_estimators']],
    'max_depth': [None, 10, 20, 30][best['max_depth']],
    'min_samples_split': [2, 5, 10][best['min_samples_split']]
}
bayes_model = RandomForestClassifier(**bayes_params, random_state=42)
bayes_model.fit(X_train, y_train)
bayes_pred = bayes_model.predict(X_test)

print("Test accuracy with Bayesian Optimization:", accuracy_score(y_test, bayes_pred))


Test accuracy with GridSearchCV: 1.0
Test accuracy with RandomizedSearchCV: 1.0
Test accuracy with Bayesian Optimization: 1.0
